In [1]:
from torch_snippets import *

In [2]:
import wget

In [5]:
output_directory = 'D:\datasets'
url = 'https://storage.googleapis.com/openimages/v5/train-annotations-object-segmentation.csv'
wget.download(url, out=output_directory)

'D:\\datasets/train-annotations-object-segmentation.csv'

In [6]:
output_directory = 'D:\datasets'
url = 'https://raw.githubusercontent.com/openimages/dataset/master/dict.csv'
wget.download(url, out=output_directory)

'D:\\datasets/dict.csv'

In [3]:
# specify the classes we want our model to predict
required_classes = ['person', 'dog', 'bird', 'car', 'elephant', 'football', 'jug', 'laptop', 'Mushroom', 'Pizza', 'Rocket', 'Shirt', 'Traffic sign', 'Watermelon', 'Zebra']
required_classes = [c.lower() for c in required_classes]


In [4]:
import pandas as pd

In [5]:
classes = pd.read_csv('D:\datasets\classes.csv', header=None)
classes.columns = ['class', 'class_name']
classes = classes[classes['class_name'].map(lambda x: x in required_classes ) ]

In [6]:
classes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 43 to 7647
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   class       15 non-null     object
 1   class_name  15 non-null     object
dtypes: object(2)
memory usage: 360.0+ bytes


In [7]:
classes

,class,class_name
43,/m/01226z,football
224,/m/015p6,bird
601,/m/01c648,laptop
753,/m/01g317,person
1022,/m/01mqdt,traffic sign
1045,/m/01n4qj,shirt
2819,/m/02vx4,football
4675,/m/052sf,mushroom
5181,/m/0663v,pizza
5916,/m/0898b,zebra


In [8]:
from torch_snippets import *

In [4]:
# reading a large file in chunks and appending to a dataframe
def read_csv_chunks(path, chunksize=1000):
    df = pd.DataFrame()
    for chunk in pd.read_csv(path, chunksize=chunksize):
        df = df.append(chunk)
    return df

In [9]:
# df = pd.read_csv('D:/Datasets/annotations.csv', chunksize=1)
df = pd.read_csv('D:/Datasets/annotations.csv')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2686666 entries, 0 to 2686665
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   MaskPath      object 
 1   ImageID       object 
 2   LabelName     object 
 3   BoxID         object 
 4   BoxXMin       float64
 5   BoxXMax       float64
 6   BoxYMin       float64
 7   BoxYMax       float64
 8   PredictedIoU  float64
 9   Clicks        object 
dtypes: float64(5), object(5)
memory usage: 205.0+ MB


In [11]:
data  = pd.merge(df, classes, left_on='LabelName', right_on='class')

In [12]:
# fetching fewer images for each class
subset_data = data.groupby('class_name').agg({'ImageID': lambda x: list(x)[:1000]})  # fetching 1000 images for each class
subset_data = flatten(subset_data.ImageID.tolist())
subset_data = data[data['ImageID'].map(lambda x: x in subset_data)]
subset_masks = subset_data['MaskPath'].tolist()

In [13]:
subset_masks[1:10]

['18c1fd8342d55af8_m01g317_a9a97702.png',
 '42ea7facefea9a28_m01g317_333376e6.png',
 'bcb123bf7f256b1c_m01g317_a42b01bc.png',
 'a2bb5944e071223c_m01g317_7ef4ab8d.png',
 '4fd2304f1f65dc74_m01g317_336e67e1.png',
 '0113dbe2b087bd55_m01g317_d606a644.png',
 'cc8c1cec5a263473_m01g317_5dd13cd7.png',
 'd6dc04e65df11e78_m01g317_cfc9e8e5.png',
 'db35878a7d398f65_m01g317_d61cc87b.png']

In [14]:
subset_data

,MaskPath,ImageID,LabelName,BoxID,BoxXMin,BoxXMax,BoxYMin,BoxYMax,PredictedIoU,Clicks,class,class_name
0,e9a7da5ff1fd56df_m01g317_1803811b.png,e9a7da5ff1fd56df,/m/01g317,1803811b,0.468125,0.628750,0.188714,0.949121,0.86841,0.47513 0.38255 1;0.52118 0.91333 0;0.47513 0....,/m/01g317,person
1,18c1fd8342d55af8_m01g317_a9a97702.png,18c1fd8342d55af8,/m/01g317,a9a97702,0.394375,0.605000,0.241667,0.903333,0.81242,0.57036 0.81358 1;0.54599 0.65655 0;0.42280 0....,/m/01g317,person
2,42ea7facefea9a28_m01g317_333376e6.png,42ea7facefea9a28,/m/01g317,333376e6,0.791875,0.839375,0.096226,0.184906,0.64984,0.79876 0.17372 1;0.84756 0.19868 1;0.80936 0....,/m/01g317,person
3,bcb123bf7f256b1c_m01g317_a42b01bc.png,bcb123bf7f256b1c,/m/01g317,a42b01bc,0.478125,0.518750,0.450797,0.604499,0.77027,0.49924 0.57654 0;0.50731 0.45367 1;0.49108 0....,/m/01g317,person
4,a2bb5944e071223c_m01g317_7ef4ab8d.png,a2bb5944e071223c,/m/01g317,7ef4ab8d,0.738750,0.861250,0.508333,0.631667,0.71815,0.79186 0.54778 0;0.80219 0.52627 1;0.84317 0....,/m/01g317,person
...,...,...,...,...,...,...,...,...,...,...,...,...
226635,19b9b7acfefbd162_m08hvt4_1682655d.png,19b9b7acfefbd162,/m/08hvt4,1682655d,0.631250,0.928750,0.296667,0.802500,0.90699,0.91454 0.35031 0;0.92895 0.53163 0;0.93898 0....,/m/08hvt4,jug
226636,7d9fd88b2699cc6d_m08hvt4_d785dbd9.png,7d9fd88b2699cc6d,/m/08hvt4,d785dbd9,0.513125,0.659375,0.298311,0.608818,0.81158,0.65518 0.33896 1;0.62441 0.44468 0;0.65074 0....,/m/08hvt4,jug
226637,1c5f7fc01eea9b1a_m08hvt4_2f96d800.png,1c5f7fc01eea9b1a,/m/08hvt4,2f96d800,0.355000,0.511250,0.579737,0.881801,0.84473,0.46071 0.64206 1;0.50700 0.64289 1;0.37250 0....,/m/08hvt4,jug
226638,6085ff0ae91fb440_m08hvt4_4aeb7989.png,6085ff0ae91fb440,/m/08hvt4,4aeb7989,0.344278,0.651032,0.683750,0.901250,0.84358,0.35302 0.76069 1;0.39130 0.72297 0;0.41286 0....,/m/08hvt4,jug


In [32]:
wget.download('https://storage.googleapis.com/openimages/v5/train-masks/train-masks-4.zip', out='D:/datasets/detection/masks')

'D:/datasets/detection/masks/train-masks-4.zip'

In [33]:
wget.download('https://storage.googleapis.com/openimages/v5/train-masks/train-masks-5.zip', out='D:/datasets/detection/masks')
wget.download('https://storage.googleapis.com/openimages/v5/train-masks/train-masks-6.zip', out='D:/datasets/detection/masks')
wget.download('https://storage.googleapis.com/openimages/v5/train-masks/train-masks-7.zip', out='D:/datasets/detection/masks')
wget.download('https://storage.googleapis.com/openimages/v5/train-masks/train-masks-8.zip', out='D:/datasets/detection/masks')
wget.download('https://storage.googleapis.com/openimages/v5/train-masks/train-masks-9.zip', out='D:/datasets/detection/masks')
wget.download('https://storage.googleapis.com/openimages/v5/train-masks/train-masks-a.zip', out='D:/datasets/detection/masks')
wget.download('https://storage.googleapis.com/openimages/v5/train-masks/train-masks-b.zip', out='D:/datasets/detection/masks')
wget.download('https://storage.googleapis.com/openimages/v5/train-masks/train-masks-c.zip', out='D:/datasets/detection/masks')
wget.download('https://storage.googleapis.com/openimages/v5/train-masks/train-masks-d.zip', out='D:/datasets/detection/masks')
wget.download('https://storage.googleapis.com/openimages/v5/train-masks/train-masks-e.zip', out='D:/datasets/detection/masks')
wget.download('https://storage.googleapis.com/openimages/v5/train-masks/train-masks-f.zip', out='D:/datasets/detection/masks')

'D:/datasets/detection/masks/train-masks-f.zip'

In [34]:
os.listdir('D:/datasets/detection/masks')

['train-masks-0.zip',
 'train-masks-1.zip',
 'train-masks-2.zip',
 'train-masks-3.zip',
 'train-masks-4.zip',
 'train-masks-5.zip',
 'train-masks-6.zip',
 'train-masks-7.zip',
 'train-masks-8.zip',
 'train-masks-9.zip',
 'train-masks-a.zip',
 'train-masks-b.zip',
 'train-masks-c.zip',
 'train-masks-d.zip',
 'train-masks-e.zip',
 'train-masks-f.zip']

In [35]:
# unzip the files
import zipfile
import os
# path to the folder with all the zipped files
folder = 'D:/datasets/detection/masks'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    zip_ref = zipfile.ZipFile(file_path, 'r')
    zip_ref.extractall(folder)
    zip_ref.close()

In [36]:
# delete the zip files, to save space
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    if file_path.endswith('.zip'):
        os.remove(file_path)

In [15]:
# get the list of all the masks
tmp_masks = Glob('D:/datasets/detection/masks/*', silent=True)

In [16]:
tmp_masks[1:5]

[» D:\datasets\detection\masks\0000048549557964_m0k4j_1232f6e4.png,
 » D:\datasets\detection\masks\0000048549557964_m0k4j_22dcf663.png,
 » D:\datasets\detection\masks\0000048549557964_m0k4j_d5cba825.png,
 » D:\datasets\detection\masks\0000048549557964_m0pg52_93cba453.png]

In [17]:
# only file name from windows path object
tmp_masks_file_name = [os.path.basename(oldname) for oldname in tmp_masks]

In [18]:
tmp_masks_file_name[0:4]

['0000048549557964_m0k4j_0f7c02af.png',
 '0000048549557964_m0k4j_1232f6e4.png',
 '0000048549557964_m0k4j_22dcf663.png',
 '0000048549557964_m0k4j_d5cba825.png']

In [32]:
os.path.basename(tmp_masks[0])

'0000048549557964_m0k4j_0f7c02af.png'

In [29]:
subset_masks[0:2]

['e9a7da5ff1fd56df_m01g317_1803811b.png',
 '18c1fd8342d55af8_m01g317_a9a97702.png']

In [22]:
# get the list of masks that are in the subset
# items = [(m, os.path.basename(m)) for m in tmp_masks if os.path.basename(m) in subset_masks]

In [33]:
# get the list of masks that are in the subset
items = [m for m in tmp_masks_file_name if m in subset_masks]

In [35]:
len(items)

0

In [36]:
items[0:2]

[]

In [25]:
# rename files and transfer to a new folder
for m, f in items:
    os.rename(m, f'D:/datasets/detection/final_masks/{f}')


In [37]:
masks = Glob('D:/datasets/detection/final_masks/*', silent=True)

In [38]:
masks[0:2]

[]

In [28]:
os.path.basename(masks[0])

IndexError: list index out of range

In [ ]:
masks = [os.path.basename(m) for m in masks]

In [ ]:
masks[0:2]

['000033469fb48bc1_m01n4qj_378b9f3a.png',
 '000033469fb48bc1_m01n4qj_6ff14b4c.png']

In [ ]:
subset_data[0]

,MaskPath,ImageID,LabelName,BoxID,BoxXMin,BoxXMax,BoxYMin,BoxYMax,PredictedIoU,Clicks,class,class_name
0,e9a7da5ff1fd56df_m01g317_1803811b.png,e9a7da5ff1fd56df,/m/01g317,1803811b,0.468125,0.628750,0.188714,0.949121,0.86841,0.47513 0.38255 1;0.52118 0.91333 0;0.47513 0....,/m/01g317,person
1,18c1fd8342d55af8_m01g317_a9a97702.png,18c1fd8342d55af8,/m/01g317,a9a97702,0.394375,0.605000,0.241667,0.903333,0.81242,0.57036 0.81358 1;0.54599 0.65655 0;0.42280 0....,/m/01g317,person
2,42ea7facefea9a28_m01g317_333376e6.png,42ea7facefea9a28,/m/01g317,333376e6,0.791875,0.839375,0.096226,0.184906,0.64984,0.79876 0.17372 1;0.84756 0.19868 1;0.80936 0....,/m/01g317,person
3,bcb123bf7f256b1c_m01g317_a42b01bc.png,bcb123bf7f256b1c,/m/01g317,a42b01bc,0.478125,0.518750,0.450797,0.604499,0.77027,0.49924 0.57654 0;0.50731 0.45367 1;0.49108 0....,/m/01g317,person
4,a2bb5944e071223c_m01g317_7ef4ab8d.png,a2bb5944e071223c,/m/01g317,7ef4ab8d,0.738750,0.861250,0.508333,0.631667,0.71815,0.79186 0.54778 0;0.80219 0.52627 1;0.84317 0....,/m/01g317,person
...,...,...,...,...,...,...,...,...,...,...,...,...
226635,19b9b7acfefbd162_m08hvt4_1682655d.png,19b9b7acfefbd162,/m/08hvt4,1682655d,0.631250,0.928750,0.296667,0.802500,0.90699,0.91454 0.35031 0;0.92895 0.53163 0;0.93898 0....,/m/08hvt4,jug
226636,7d9fd88b2699cc6d_m08hvt4_d785dbd9.png,7d9fd88b2699cc6d,/m/08hvt4,d785dbd9,0.513125,0.659375,0.298311,0.608818,0.81158,0.65518 0.33896 1;0.62441 0.44468 0;0.65074 0....,/m/08hvt4,jug
226637,1c5f7fc01eea9b1a_m08hvt4_2f96d800.png,1c5f7fc01eea9b1a,/m/08hvt4,2f96d800,0.355000,0.511250,0.579737,0.881801,0.84473,0.46071 0.64206 1;0.50700 0.64289 1;0.37250 0....,/m/08hvt4,jug
226638,6085ff0ae91fb440_m08hvt4_4aeb7989.png,6085ff0ae91fb440,/m/08hvt4,4aeb7989,0.344278,0.651032,0.683750,0.901250,0.84358,0.35302 0.76069 1;0.39130 0.72297 0;0.41286 0....,/m/08hvt4,jug


In [ ]:
subset_data = subset_data[subset_data['MaskPath'].map(lambda x: x in masks)]

In [ ]:
subset_imageIds = subset_data['ImageID'].tolist()

In [ ]:
from openimages.download import _download_images_by_id

In [ ]:
_download_images_by_id(subset_imageIds, 'train', 'D:/datasets/detection/images')

100%|██████████| 23053/23053 [04:16<00:00, 89.72it/s]


In [ ]:
! pip install git+git://github.com/waspinator/pycococreator.git@0.2.0

In [39]:
import datetime

In [40]:
INFO = {
    "description": "Open Images Dataset V5",
    "url": "None",
    "version": "1.0",
    "year": 2023,
    "contributor": "JT",
    "date_created": datetime.datetime.utcnow().isoformat(' ')
}



In [113]:
LICENSES = [
    {
        "id": 1,
        "name": "MIT License",
    }
]

In [41]:
classes[0:2]

,class,class_name
43,/m/01226z,football
224,/m/015p6,bird


In [43]:
CATEGORIES = [ {'id': id+1, 'name': name.replace('/',''), 'supercategory': 'none'} for id, (_, (name, clss_name)) in enumerate(classes.iterrows())]

In [ ]:
from pycocotools import pycococreatortools